In [ ]:
import tkinter as tk
import random
import time

TAMANO_GRID = 10
NUM_BOMBAS = 10

botones_grid = {}
es_bomba = {}
juego_terminado = False
casillas_reveladas = 0 
inicio = 0
etiqueta_cronometro = None
ventana = None
marco_opciones = None
marco_juego = None 
marco_etiquetas = None 

def contar_bombas_cercanas(r, c):
    "Cuenta el número de bombas en el radio 3x3 alrededor de (r, c)."
    conteo = 0 # cuenta cuantas bombas hay al rededor
    for filas in range(-1, 2): # itera sobre la casilla seleccionada a la izquierda y a derecha 
        for columnas in range(-1, 2): # itera sobre la casilla seleccionada de la casilla de arriba y abajo
            if filas == 0 and columnas == 0:
                continue
            
            fila_vecina, columna_vecina = r + filas, c + columnas

            if 0 <= fila_vecina < TAMANO_GRID and 0 <= columna_vecina < TAMANO_GRID:
                if es_bomba.get((fila_vecina, columna_vecina), False):
                    conteo = conteo + 1
    return conteo

def colocar_bandera(r, c):
    """Se activa con el clic derecho para poner o quitar banderas."""
    
    if juego_terminado:
        return

    boton_especifico = botones_grid.get((r, c))

  
    if not boton_especifico or boton_especifico['state'] == 'disabled':
        return

  
    if boton_especifico['text'] == " ":
        boton_especifico.config(text="🚩", fg="red")
    elif boton_especifico['text'] == "🚩":
        boton_especifico.config(text=" ", fg="black")

def casilla_presionada(r, c):
    global juego_terminado, casillas_reveladas
    
    if juego_terminado:
        return 

    boton_especifico = botones_grid.get((r, c))
    
    if not boton_especifico or boton_especifico['state'] == 'disabled':
        return

   
    if boton_especifico['text'] == "🚩":
        return

    if es_bomba[(r, c)]:
        juego_terminado = True
        for (fila, col), boton in botones_grid.items():
            if es_bomba[(fila, col)]:
                boton.config(text="💣", bg="red", state="disabled", relief="sunken")
            else:
                boton.config(state="disabled", relief="sunken")
        boton_especifico.config(text="💣", bg="red", state="disabled", relief="sunken")

    else:
        casillas_reveladas += 1 
        boton_especifico.config(state="disabled", relief="sunken", bg="lightgray")
        
        conteo_bombas = contar_bombas_cercanas(r, c)
        
        if conteo_bombas > 0:
            boton_especifico.config(text=str(conteo_bombas))
        else:
            boton_especifico.config(text="") 
            for filas in range(-1, 2):
                for columnas in range(-1, 2):
                    if filas == 0 and columnas == 0:
                        continue
                    fila_vecina, columna_vecina = r + filas, c + columnas
                    if 0 <= fila_vecina < TAMANO_GRID and 0 <= columna_vecina < TAMANO_GRID:
                        casilla_presionada(fila_vecina, columna_vecina)
        
        
        total_casillas_seguras = (TAMANO_GRID * TAMANO_GRID) - NUM_BOMBAS
        if casillas_reveladas == total_casillas_seguras:
            juego_terminado = True
            etiqueta_cronometro.config(text="¡Ganaste!")
            
            for (fila, col), boton in botones_grid.items():
                 if es_bomba[(fila, col)]:
                    boton.config(text="🚩", fg="green", state="disabled", relief="sunken")


def cronometro():
    global inicio, etiqueta_cronometro
    
    if juego_terminado:        
        return 
        
    tiempo_transcurrido = time.time() - inicio
    string_tiempo = time.strftime('%M:%S', time.gmtime(tiempo_transcurrido))
    
    if etiqueta_cronometro:
        etiqueta_cronometro.config(text=f"Tiempo: {string_tiempo}")
    
    ventana.after(1000, cronometro) 

def limpiar_ventana():
    "Destruye los frames de la ventana"
    global marco_juego, marco_etiquetas
    if marco_juego: # verifica que el marco_juego existe
        marco_juego.destroy() # destruye la grilla 
    if marco_etiquetas: # verifica que el marco_etiquetas_existe
        marco_etiquetas.destroy()


def volver_al_menu():
    "Limpia el juego y muestra el menú principal."
    global juego_terminado
    juego_terminado = True 
    limpiar_ventana()
    marco_opciones.pack(expand=True)
   
    ventana.geometry("400x500+480+0") 


def iniciar_juego():
    global inicio, juego_terminado, botones_grid, es_bomba, etiqueta_cronometro
    global marco_juego, marco_etiquetas, casillas_reveladas
    
    limpiar_ventana()

    botones_grid = {}
    es_bomba = {}
    juego_terminado = False
    casillas_reveladas = 0 
    
    
    marco_etiquetas = tk.Frame(ventana)
    marco_etiquetas.pack(pady=5) 

 
    boton_menu = tk.Button(marco_etiquetas, text="Menú", font=("Arial", 10), command=volver_al_menu)
    boton_menu.pack(side=tk.LEFT, padx=20)

   
    boton_reiniciar = tk.Button(marco_etiquetas, text="⟳", font=("Arial", 14), command=iniciar_juego)
    boton_reiniciar.pack(side=tk.LEFT, padx=10)   

    etiqueta_cronometro = tk.Label(marco_etiquetas, text="Tiempo: 00:00", font=("Arial", 14), width=15)
    etiqueta_cronometro.pack(side=tk.LEFT, padx=20)
    
 
    marco_juego = tk.Frame(ventana)
    marco_juego.pack(pady=10, padx=10) 

    for i in range(TAMANO_GRID):
        for j in range(TAMANO_GRID):
            boton = tk.Button(marco_juego, text=" ", width=3, height=2,
                             bg="#c0c0c0", activebackground="#e0e0e0", relief="raised")
            boton.grid(column=j, row=i)
            botones_grid[(i, j)] = boton
            es_bomba[(i, j)] = False
           
            boton.config(command=lambda r=i, c=j: casilla_presionada(r, c))
            # Clic derecho (<Button-3>)
            boton.bind("<Button-3>", lambda e, r=i, c=j: colocar_bandera(r, c))

    coordenadas_posibles = list(botones_grid.keys())
    num_bombas_real = min(NUM_BOMBAS, len(coordenadas_posibles))
    coordenadas_bombas = random.sample(coordenadas_posibles, num_bombas_real)

    for (r, c) in coordenadas_bombas:
        es_bomba[(r, c)] = True
    
    inicio = time.time()
    cronometro()

def seleccionar_dificultad_y_jugar(tamano, bombas):
    """Establece la dificultad, ajusta la ventana y comienza el juego"""
    global TAMANO_GRID, NUM_BOMBAS

    TAMANO_GRID = tamano
    NUM_BOMBAS = bombas

    if NUM_BOMBAS == 10: # 10x10
        ventana.geometry("380x480+1100+0") 
        ventana.minsize(380, 480)
    elif NUM_BOMBAS == 12: # 12x12
        ventana.geometry("450x550+680+0")
        ventana.minsize(450, 550)
    elif NUM_BOMBAS == 20: # 15x15
        ventana.geometry("550x650+400+0")
        ventana.minsize(550, 650)

    marco_opciones.pack_forget()
    iniciar_juego()


ventana = tk.Tk()
ventana.title("Buscaminas")
ventana.geometry("400x500+480+0") 
ventana.minsize(300, 400)
# ventana.iconbitmap("Buscaminas-meme.ico") 


marco_opciones = tk.Frame(ventana, pady=50)
marco_opciones.pack(expand=True)

etiqueta_titulo = tk.Label(marco_opciones, text="BUSCAMINAS", font=("Arial", 24, "bold"))
etiqueta_titulo.pack(pady=20)

etiqueta_subtitulo = tk.Label(marco_opciones, text="Selecciona una dificultad:", font=("Arial", 14))
etiqueta_subtitulo.pack(pady=10)

estilo_boton = {"font": ("Arial", 12), "width": 25, "pady": 10, "bg": "#d0d0d0", "relief": "raised"}

boton_facil = tk.Button(marco_opciones, text="Principiante (10x10, 10 bombas)", **estilo_boton,
                        command=lambda: seleccionar_dificultad_y_jugar(10, 10))
boton_facil.pack(pady=5)

boton_medio = tk.Button(marco_opciones, text="Intermedio (12x12, 12 bombas)", **estilo_boton,
                        command=lambda: seleccionar_dificultad_y_jugar(12, 12))
boton_medio.pack(pady=5)

boton_dificil = tk.Button(marco_opciones, text="Experto (15x15, 20 bombas)", **estilo_boton,
                          command=lambda: seleccionar_dificultad_y_jugar(15, 20))
boton_dificil.pack(pady=5)

ventana.iconbitmap("buscaminas-meme.ico")

ventana.mainloop()
